## Coroutines


### Basic behavior of a generator used as a coroutine


In [1]:
# example 16-1
def simple_coroutine():
    print('-> coroutine start')
    x = yield
    print('-> coroutine receive: ', x)
    


In [2]:
my_cor = simple_coroutine()
my_cor

<generator object simple_coroutine at 0x7fa0896cca98>

In [3]:
next(my_cor)

-> coroutine start


In [4]:
my_cor.send(40)


-> coroutine receive:  40


StopIteration: 

A coroutine can be in one of four states. You can determine the current state using the
inspect.getgeneratorstate(...) function which returns one of these strings:

'GEN_CREATED'

Waiting to start execution.

'GEN_RUNNING'

Currently being executed by the interpreter1

'GEN_SUSPENDED'

Currently suspended at a yield expression.

'GEN_CLOSED'

Execution has completed.


In [6]:
my_c = simple_coroutine()
my_c.send(1234)


TypeError: can't send non-None value to a just-started generator

In [11]:
# Example 16-2. A coroutine that yields twice.
from inspect import getgeneratorstate

def simple_coro2(a):
    print('--> coroutine start')
    b = yield a
    print('--> received: b = ', b)
    c = yield a + b
    print('--> received: c = ', c)
    
    

In [12]:
my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [13]:
next(my_coro2)

--> coroutine start


14

In [14]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [15]:
my_coro2.send(28)

--> received: b =  28


42

In [16]:
my_coro2.send(999)

--> received: c =  999


StopIteration: 

In [17]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

函数执行过程

![](Selection_005.jpg)

###  coroutine to compute a running average


In [18]:
# Example 16-3.
def averager():
    total = 0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
        

In [19]:
coro_avg = averager()
next(coro_avg)


In [20]:
coro_avg.send(10)

10.0

In [21]:
coro_avg.send(30)

20.0

上述代码的执行流程

next 将函数语句执行到 yield 之前，也就是初始化一些参数，然后开始等待 term 被赋值

当 .send(10)　执行时， term 被赋值为 10，然后接着向下执行

直到 while 

### Decorators for coroutine priming

In [1]:
# Example 16-5. coroutil.py: decorator for priming coroutine.

from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kw):
        gen = func(*args, **kw)
        next(gen)
        return gen
    
    return primer



In [2]:

@coroutine
def averager():
    total = 0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [3]:
from inspect import getgeneratorstate

avg = averager()
getgeneratorstate(avg)

'GEN_SUSPENDED'

In [4]:
# 以上结果是由于　 coroutine 装饰器的结果
# 不是 WAIT 状态， 而是 SUSPENDED 状态

avg.send(10)

10.0

In [5]:
avg.send(30)

20.0

In [9]:
# generator.throw
# generator.close

class DemoException(Exception):
    '''an exception type for the demonstration'''
    
@coroutine
def demo_exc_handling():
    print('--> coroutine start')
    while True:
        try:
            x = yield 
        except DemoException:
            print(' *** DemoException handled continuing...')
        else:
            print('--> coroutine received : {!r}'.format(x))
            
    raise RuntimeError('this line should never run')

In [10]:
demo = demo_exc_handling()

demo.send(10)

--> coroutine start
--> coroutine received : 10


In [11]:
demo.send(20)

--> coroutine received : 20


In [12]:
demo.close()

In [13]:
getgeneratorstate(demo)

'GEN_CLOSED'

In [14]:
demo2 = demo_exc_handling()


--> coroutine start


In [15]:
demo2.throw(DemoException)

 *** DemoException handled continuing...


In [16]:
getgeneratorstate(demo2)

'GEN_SUSPENDED'

In [17]:
demo2.send(2)

--> coroutine received : 2


In [18]:
# 因为程序捕捉到了 demoexception 错误
# 所以 gen 的状态是 suspend 
# 可以继续往后运行
# 当给到一个　除零异常　时，gen 的状态会自动关闭

demo2.throw(ZeroDivisionError)

ZeroDivisionError: 

In [19]:
getgeneratorstate(demo2)

'GEN_CLOSED'

### Returning a value from a coroutine


In [26]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

@coroutine
def averager():
    
    total = 0
    count = 0
    average = None
    
    while True:
        
        term = yield 
        if not term: break
        total += term
        count += 1
        average = total / count 
        
    return Result(count, average)



In [28]:
avg_ = averager()
avg_.send(10)

In [29]:
avg_.send(20)

In [30]:
avg_.send(None)

StopIteration: Result(count=2, average=15.0)

### Using yield from


In [31]:
# new in python3.3

def gen():
    
    yield from 'AB'
    yield from range(3)
    
list(gen())

['A', 'B', 0, 1, 2]

In [32]:
def chain(*iterables):
    for i in iterables:
        yield from i
        
s = 'ABC'
b = range(3)

list(chain(s,b))

['A', 'B', 'C', 0, 1, 2]

In [38]:
# 关于 yield from 完整的一个例子

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# the subgenerator
def averager():
    
    total = 0
    count = 0
    average = None
    
    while True:
        term = yield 
        if not term: break
        total += term
        count += 1
        average = total / count
        
    return Result(count, average)


# the delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
        
# the client code, a.k.a the caller
def main(data):
    results = {}
    for key, val in data.items():
        group = grouper(results, key)
        next(group)
        for v in val:
            group.send(v)
        group.send(None)
        
    report(results)

# output report
def report(results):
    for key, val in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2}{:5} averaging {:2f}{}'.format(val.count, group, val.average, unit))
        
data = {
'girls;kg':
[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
'girls;m':
[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
'boys;kg':
[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
'boys;m':
[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

if __name__ == '__main__':
    main(data)
    
    

 9boys  averaging 40.422222kg
 9boys  averaging 1.388889m
10girls averaging 42.040000kg
10girls averaging 1.428000m


### The meaning of yield from


In [4]:
# n Example 16-18.
# 伪代码
EXPR = list(range(10))

def func():
    _i = iter(EXPR)
    try:
        _y = next(_i)
    except StopIteration as e:
        _r = _e.value
    else:
        while 1:
            _s = yield _y
            try:
                _y = _i.send(_s)
            except StopIteration as _e:
                _r = _e.value
                break

    RESULT = _r
    return RESULT

# 以上伪代码相当于

def func2():
    RESULT = yield from EXPR


In [7]:
next(func())

0

In [9]:
next(func2())

0

In [12]:
print(next(func()))
print(next(func()))
print(next(func()))

0
0
0


### Use case: coroutines for discrete event simulation

In [13]:
# 出租车载客模拟

from collections import namedtuple

Event = namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    '''yield to simulator issuing event at each state change'''
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
        
    yield Event(time, ident, 'going home')
    


In [14]:
zxt = taxi_process(1, 2, 0)
next(zxt)

Event(time=0, proc=1, action='leave garage')

In [20]:
# _ 变量表示上一个结果，其实质是一个 namedtuple 
zxt.send(_.time + 7)

Event(time=19, proc=1, action='drop off passenger')

In [16]:
zxt.send(5)

Event(time=5, proc=1, action='drop off passenger')

In [19]:
zxt.send(_.time + 7)

Event(time=12, proc=1, action='pick up passenger')

In [27]:
import queue
import random

class Simulator:
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of event ***')
                break
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi: ', proc_id, proc_id*'  ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events ending'
            print(msg.format(self.events.qsize()))
            
        
def compute_duration(event):
    if event == 'leave garage':
        return random.choice(range(1,3))
    elif event == 'pick up passenger':
        return random.choice(range(3,5))
    elif event == 'drop off passenger':
        return random.choice(range(2,4))
    else:
        return 0
    
    

                              

In [28]:
taxis = {i: taxi_process(i, (i+1)*2, i*2) for i in range(3)}
sim = Simulator(taxis)
sim.run(100)

taxi:  0  Event(time=0, proc=0, action='leave garage')
taxi:  0  Event(time=1, proc=0, action='pick up passenger')
taxi:  1    Event(time=2, proc=1, action='leave garage')
taxi:  1    Event(time=4, proc=1, action='pick up passenger')
taxi:  2      Event(time=4, proc=2, action='leave garage')
taxi:  0  Event(time=5, proc=0, action='drop off passenger')
taxi:  2      Event(time=5, proc=2, action='pick up passenger')
taxi:  0  Event(time=7, proc=0, action='pick up passenger')
taxi:  1    Event(time=7, proc=1, action='drop off passenger')
taxi:  2      Event(time=8, proc=2, action='drop off passenger')
taxi:  1    Event(time=9, proc=1, action='pick up passenger')
taxi:  0  Event(time=10, proc=0, action='drop off passenger')
taxi:  2      Event(time=11, proc=2, action='pick up passenger')
taxi:  1    Event(time=12, proc=1, action='drop off passenger')
taxi:  0  Event(time=13, proc=0, action='going home')
taxi:  1    Event(time=14, proc=1, action='pick up passenger')
taxi:  2      Event(time